To implement the encodeRS(m) function for a [255, 128] Reed Solomon code with generator polynomial g(x) = (x-1)(x-α)(x-α^2)*(x-α^3)...(x-α^127), we can use the following steps:

    Convert the input message m into a polynomial m(x) of degree k-1, where k = 128, by setting m(x) = m_0 + m_1x + ... + m_k-1x^(k-1).

    Pad the message polynomial m(x) with k = 128 zeros to form a new polynomial M(x) of degree n-1 = 254, where n = 255. That is, set M(x) = m(x)*x^k.

    Calculate the remainder polynomial R(x) = M(x) mod g(x) using polynomial long division or the Euclidean algorithm in GF(2^8).

    The codeword c can be formed by appending the coefficients of the remainder polynomial R(x) to the message polynomial m(x), i.e., set c(x) = m(x) + R(x), and then evaluate the resulting polynomial c(x) at each element of GF(2^8) to obtain the codeword c = (c_0, c_1, ..., c_n-1).

In [464]:
# d = floor(128-1 / 2) = 63 
# i.e correct up to 63 errors 

In [465]:
# Set the base field size to 2 (binary field)
q = 2 
# Set the extension field size to 8
m = 8 
# Define the modulus polynomial for the extension field
p = x^8 + x^4 + x^3 + x^2 + 1
# Create the extension field with the specified base field size and modulus polynomial
# The field element is named 'a'
F.<a> = GF(q^m, modulus=p) 
# Create a polynomial ring over the extension field 'F'
# The polynomial is named 'x'
R.<x> = PolynomialRing(F) 
# Set the length of the message in bytes
message_length = 128 # bytes
# Set the number of symbols in the codeword
n = 255
# Set the number of message symbols in the codeword
k = 128

In [466]:
def text2bin(msg):
    """Converts text to binary string"""
    return "".join([f"{ord(m):08b}" for m in msg])

def bin2text(bin_s):
    """Converts binary string to text"""
    return "".join([chr(int(bin_s[i: i+8], 2)) for i in range(0, len(bin_s), 8)])

def to_field_element(string):
    """Converts binary string to a field element"""
    elem = 0
    n = len(string)
    for i in range(n):
        coeff = int(string[n-i-1])
        elem += coeff * a^i # coeff is 0 or 1
    return elem

def from_field_element(field_element):
    """Converts a field element to a binary string"""
    return ''.join(map(str, vector(field_element)[::-1]))

def convert_str_to_polynomial(msg):
    """Converts a string to a polynomial in the extension field"""
    # Padding the message with zeros to make it of length k
    msg = "0"*(k - len(msg)) + msg
    # Converting the message to binary string
    message = text2bin(msg)
    # Splitting the binary string into 8-bit segments
    message = [message[i:i+8] for i in range(0, k*8, 8)]
    # Converting each 8-bit segment to a field element and multiplying it with x raised to a power
    message = sum([to_field_element(e)*x^i for i,e in enumerate(message)])
    return message


In [472]:
# define generator poly 
# g_x = prod([(x - (a^i)) for i in range((q^m / 2))])

# Define generator polynomial as the product of (x - alpha^i) for i in [0, (q^m / 2))
# where alpha is a primitive element of the finite field GF(q^m)
# and q and m are constants defined earlier in the code
g_x = prod([(x - (a^i)) for i in range((q^m / 2))])


In [473]:
def encodeRS(g, m): 
    """
    Takes a 128 - byte messagee, and encode it to a codeword 
        c, with a systematic encoding as 
        m = (m_0,m_1,...,m_k-1) => c = (m_0, m_1,...,m_k-1, c_k,..._c_n-1)
    """
    m = convert_str_to_polynomial(m)
    return m * x^(127) - (m* x^(127)) % g

In [479]:
def encode_RS(generator_poly, message):
    """
    Encodes a 128-byte message into a codeword using a systematic encoding.
    The message is first converted into a polynomial, then multiplied by x^(n-k),
    where n is the length of the codeword and k is the length of the message.
    The resulting polynomial is then divided by the generator polynomial to obtain the
    error-correction code, which is appended to the message to form the codeword.

    :param generator_poly: the generator polynomial for the Reed-Solomon code
    :param message: the message to be encoded
    :return: the encoded codeword
    """
    message_poly = convert_str_to_polynomial(message)
    codeword_poly = message_poly * x^(n-k) - (message_poly * x^(n-k)) % generator_poly
    return codeword_poly

In [481]:
# Now, my custom text is the old joke: 
# This is to be encoded, "simulated" transmissioned over the air 
# Some error is added to the polynomial 
# and need to be corrected. 
text = "Why don't scientists trust atoms? Because they make up everything! haha"
text_padding = (n - len(text)) # nice to have in this synthetic exercise 

encoded_text = encodeRS(g_x, text) # + x^4 + x + 1
r = encoded_text + x^4 + (a^7)*x + 2*x +1 # add some error vector 


In [485]:
def EEA(a, s_x, p): #Extended Euclidean Algoritm
        old_r = a;
        r = s_x;
        buffer_r = old_r;
        old_s = 1;
        s = 0;
        buffer_s = old_s;
        old_t = 0;
        t = 1;
        buffer_t = old_t;
        while old_r.degree() >= p:
            q = old_r // r;
            old_r = r;
            r = buffer_r - (q*r);
            old_s = s;
            s = buffer_s - (q*s);
            old_t = t;
            t = buffer_t - (q*t);
            buffer_r = old_r;
            buffer_s = old_s;
            buffer_t = old_t;
        return old_r, old_s, old_t


def decode_to_ascii(poly):     
    tmp = ''.join([from_field_element(_) for _ in poly])
    return (bin2text(tmp))
    
# Decoding part
def decodeRS(r): 
    # calulate syndroms 
    t = floor((n-k) / 2) 
    syndrom_list = [r(a^i) * x^(i-1) for i in range(1, 2*t+1)]
    s_x = sum([r(a^i)*x^(i-1) for i in range(1, 2*t+1)])
    if any(s_x): 
        # Not error free, doing decoding!
        omega_x, theta, lambda_ = EEA(x^(2*t), s_x, t)
        
        if lambda_.degree() >= 63: 
            print("Failure of decoding! ")
            return 0 
        
        XXX = lambda_.roots()
        deriv_ = diff(lambda_)
        
        # err locations
        e_i = [(n - y[0].log(a))%n for y in XXX]

        # error values at locations 
        err_poly = sum([((omega_x(y))/deriv_(y))*x^y for y in e_i])
        
        # error polynomials 
        # err_poly = sum([err for err in err_values])
        correct_poly = r - err_poly
        
        # decode to ascii: 
        ascii_ = decode_to_ascii(correct_poly)
        return correct_poly, ascii_
    else: # error free received vector 
        print("No errors in received R!")
        return r
    
decoded, decoded_text = decodeRS(r)

In [486]:
print(decoded_text[text_padding:])

Why don't scientists trust atoms? Because they make up everything! haha
